In [8]:
# 加载本代码所依赖的通用外部包
import json
from datasets import load_dataset
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer

In [9]:
# 加载数据集
ccpm_train = load_dataset(path="./data_datasets_CCPM/", data_files= "train.jsonl")

Using custom data configuration data_datasets_CCPM-2e3f28e187745035
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-2e3f28e187745035/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# 测试代码
print(type(ccpm_train))

<class 'datasets.dataset_dict.DatasetDict'>


In [11]:
# 测试代码
print(len(ccpm_train["train"]))

21778


In [12]:
# 测试代码
ccpm_train["train"][0]

{'translation': '诗人啊，你竟像在遥远的地方站立船头。',
 'choices': ['行人初上木兰舟', '骚人遥驻木兰舟', '有人独上木兰舟', '行人迢递木兰舟'],
 'answer': 1}

In [13]:
ccpm_valid = load_dataset(path="./data_datasets_CCPM/", data_files="valid.jsonl")
ccpm_test = load_dataset(path="./data_datasets_CCPM/", data_files="test_public.jsonl")

Using custom data configuration data_datasets_CCPM-dac694de9212fde6
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-dac694de9212fde6/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration data_datasets_CCPM-7d6d99939b9cd24a
Found cached dataset json (/home/djh/.cache/huggingface/datasets/json/data_datasets_CCPM-7d6d99939b9cd24a/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# 测试代码
ccpm_train_split = []
for i in range(0, len(ccpm_train["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_train["train"][i]["translation"]}
        temp_json["choices"] = ccpm_train["train"][i]["choices"][j]
        if j == int(ccpm_train["train"][i]["answer"]):
            temp_json["answer"] = 1
        else:
            temp_json["answer"] = 0
        ccpm_train_split.append(temp_json)

    if i == 5:
        break

for i in range(0, len(ccpm_train_split)):
    print(ccpm_train_split[i])

{'translation': '诗人啊，你竟像在遥远的地方站立船头。', 'choices': '行人初上木兰舟', 'answer': 0}
{'translation': '诗人啊，你竟像在遥远的地方站立船头。', 'choices': '骚人遥驻木兰舟', 'answer': 1}
{'translation': '诗人啊，你竟像在遥远的地方站立船头。', 'choices': '有人独上木兰舟', 'answer': 0}
{'translation': '诗人啊，你竟像在遥远的地方站立船头。', 'choices': '行人迢递木兰舟', 'answer': 0}
{'translation': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。', 'choices': '绿玉觜攒鸡脑破，玄金爪擘兔心开。', 'answer': 0}
{'translation': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。', 'choices': '翅金肉白顶红麻，项糁毛青腿少瑕。', 'answer': 0}
{'translation': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。', 'choices': '头似珊瑚项班红，翅如金箔肉带黄。', 'answer': 0}
{'translation': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。', 'choices': '碧玉炅炅双目瞳，黄金拳拳两鬓红。', 'answer': 1}
{'translation': '清晨还是西北风。', 'choices': '清晨西北转', 'answer': 1}
{'translation': '清晨还是西北风。', 'choices': '河岳西来转', 'answer': 0}
{'translation': '清晨还是西北风。', 'choices': '凌晨从北固', 'answer': 0}
{'translation': '清晨还是西北风。', 'choices': '西北转银潢', 'answer': 0}
{'translation': '柴烟中红星乱闪。', 'choices': '流星紫入烟', 'answer': 0}
{'translation': '柴烟中红星乱闪。', 'choices': '红光生

In [13]:
# 测试代码
ccpm_train

DatasetDict({
    train: Dataset({
        features: ['translation', 'choices', 'answer'],
        num_rows: 21778
    })
})

In [14]:
# 测试代码
ccpm_valid

DatasetDict({
    train: Dataset({
        features: ['translation', 'choices', 'answer'],
        num_rows: 2720
    })
})

In [15]:
# 测试代码
ccpm_test

DatasetDict({
    train: Dataset({
        features: ['translation', 'choices'],
        num_rows: 2720
    })
})

In [14]:
# 数据集分裂处理
ccpm_train_split = []
for i in range(0, len(ccpm_train["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_train["train"][i]["translation"]}
        temp_json["choices"] = ccpm_train["train"][i]["choices"][j]
        if j == int(ccpm_train["train"][i]["answer"]):
            temp_json["answer"] = 1
        else:
            temp_json["answer"] = 0
        ccpm_train_split.append(temp_json)

ccpm_valid_split = []
for i in range(0, len(ccpm_valid["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_valid["train"][i]["translation"]}
        temp_json["choices"] = ccpm_valid["train"][i]["choices"][j]
        if j == int(ccpm_valid["train"][i]["answer"]):
            temp_json["answer"] = 1
        else:
            temp_json["answer"] = 0
        ccpm_valid_split.append(temp_json)

ccpm_test_split = []
for i in range(0, len(ccpm_test["train"])):
    for j in range(0, 4):
        temp_json = {"translation": ccpm_test["train"][i]["translation"]}
        temp_json["choices"] = ccpm_test["train"][i]["choices"][j]
        temp_json["answer"] = 0
        ccpm_test_split.append(temp_json)

In [17]:
# 测试代码
# 建模思路一：将翻译和候选诗行进行拼接
ccpm_train_concat = []
for i in range(0, len(ccpm_train_split)):
    temp_json = {"text": ccpm_train_split[i]["translation"] + "=" + ccpm_train_split[i]["choices"],
                 "label": ccpm_train_split[i]["answer"]}
    ccpm_train_concat.append(temp_json)
    if i == 5: break

for i in range(0, len(ccpm_train_concat)):
    print(ccpm_train_concat[i])

{'text': '诗人啊，你竟像在遥远的地方站立船头。=行人初上木兰舟', 'label': 0}
{'text': '诗人啊，你竟像在遥远的地方站立船头。=骚人遥驻木兰舟', 'label': 1}
{'text': '诗人啊，你竟像在遥远的地方站立船头。=有人独上木兰舟', 'label': 0}
{'text': '诗人啊，你竟像在遥远的地方站立船头。=行人迢递木兰舟', 'label': 0}
{'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=绿玉觜攒鸡脑破，玄金爪擘兔心开。', 'label': 0}
{'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=翅金肉白顶红麻，项糁毛青腿少瑕。', 'label': 0}


In [15]:
# 建模思路一：将翻译和候选诗行进行拼接
ccpm_train_concat = []
for i in range(0, len(ccpm_train_split)):
    temp_json = {"text": ccpm_train_split[i]["translation"] + "=" + ccpm_train_split[i]["choices"],
                 "label": ccpm_train_split[i]["answer"]}
    ccpm_train_concat.append(temp_json)

ccpm_valid_concat = []
for i in range(0, len(ccpm_valid_split)):
    temp_json = {"text": ccpm_valid_split[i]["translation"] + "=" + ccpm_valid_split[i]["choices"],
                 "label": ccpm_valid_split[i]["answer"]}
    ccpm_valid_concat.append(temp_json)

ccpm_test_concat = []
for i in range(0, len(ccpm_test_split)):
    temp_json = {"text": ccpm_test_split[i]["translation"] + "=" + ccpm_test_split[i]["choices"],
                 "label": ccpm_test_split[i]["answer"]}
    ccpm_test_concat.append(temp_json)

In [19]:
ccpm_test_concat

[{'text': '侍奉夫主，不能尽自己的天年。=事主不尽年', 'label': 0},
 {'text': '侍奉夫主，不能尽自己的天年。=事君期尽年', 'label': 0},
 {'text': '侍奉夫主，不能尽自己的天年。=事主不及终', 'label': 0},
 {'text': '侍奉夫主，不能尽自己的天年。=万年不尽意', 'label': 0},
 {'text': '触痛了山南蜀客的似箭归心。=前山蜀客归', 'label': 0},
 {'text': '触痛了山南蜀客的似箭归心。=前溪有归客', 'label': 0},
 {'text': '触痛了山南蜀客的似箭归心。=蜀客亦思归', 'label': 0},
 {'text': '触痛了山南蜀客的似箭归心。=青山待客归', 'label': 0},
 {'text': '为王事奔波路程尚没有走尽。=王程应未尽', 'label': 0},
 {'text': '为王事奔波路程尚没有走尽。=王程尚未安', 'label': 0},
 {'text': '为王事奔波路程尚没有走尽。=王程余未尽', 'label': 0},
 {'text': '为王事奔波路程尚没有走尽。=王程未可稽', 'label': 0},
 {'text': '可惜到了黄昏，娇艳芳香的花儿凋零。=日暮槿花零落风', 'label': 0},
 {'text': '可惜到了黄昏，娇艳芳香的花儿凋零。=可怜日暮嫣香落', 'label': 0},
 {'text': '可惜到了黄昏，娇艳芳香的花儿凋零。=可怜日落长安路', 'label': 0},
 {'text': '可惜到了黄昏，娇艳芳香的花儿凋零。=可怜日暮蔫香落', 'label': 0},
 {'text': '是你又狂放又侠义又温文尔雅。=亦狂亦侠亦温文', 'label': 0},
 {'text': '是你又狂放又侠义又温文尔雅。=狂侠何妨亦粹温', 'label': 0},
 {'text': '是你又狂放又侠义又温文尔雅。=不饮亦狂饮亦狂', 'label': 0},
 {'text': '是你又狂放又侠义又温文尔雅。=亦文亦质洵彬然', 'label': 0},
 {'text': '我本来出身良家，懂得许多门道。=我本小家女', 'labe

In [20]:
# 测试代码
ccpm_train_concat

[{'text': '诗人啊，你竟像在遥远的地方站立船头。=行人初上木兰舟', 'label': 0},
 {'text': '诗人啊，你竟像在遥远的地方站立船头。=骚人遥驻木兰舟', 'label': 1},
 {'text': '诗人啊，你竟像在遥远的地方站立船头。=有人独上木兰舟', 'label': 0},
 {'text': '诗人啊，你竟像在遥远的地方站立船头。=行人迢递木兰舟', 'label': 0},
 {'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=绿玉觜攒鸡脑破，玄金爪擘兔心开。', 'label': 0},
 {'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=翅金肉白顶红麻，项糁毛青腿少瑕。', 'label': 0},
 {'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=头似珊瑚项班红，翅如金箔肉带黄。', 'label': 0},
 {'text': '他的双眼眼瞳碧绿而有光，头发金黄而弯曲，两鬓呈红色。=碧玉炅炅双目瞳，黄金拳拳两鬓红。', 'label': 1},
 {'text': '清晨还是西北风。=清晨西北转', 'label': 1},
 {'text': '清晨还是西北风。=河岳西来转', 'label': 0},
 {'text': '清晨还是西北风。=凌晨从北固', 'label': 0},
 {'text': '清晨还是西北风。=西北转银潢', 'label': 0},
 {'text': '柴烟中红星乱闪。=流星紫入烟', 'label': 0},
 {'text': '柴烟中红星乱闪。=红光生紫烟', 'label': 0},
 {'text': '柴烟中红星乱闪。=乱荷红带紫', 'label': 0},
 {'text': '柴烟中红星乱闪。=红星乱紫烟', 'label': 1},
 {'text': '在他们身边痛哭的只有尚未省事的儿郎。=狂叫唯童儿', 'label': 1},
 {'text': '在他们身边痛哭的只有尚未省事的儿郎。=狂呼造化儿', 'label': 0},
 {'text': '在他们身边痛哭的只有尚未省事的儿郎。=儿童趁欲狂', 'label': 0},
 {'text': '在他们身边痛哭的只有尚未省事的儿

In [18]:
# 预处理1-3：
# 1.加载句子标记器以处理字段：text
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [19]:
# 2.定义预处理函数
# 用于标记和截断序列，使其不超过模型的最大输入长度：text
def preproccess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [20]:
# 3.对整个数据集进行批量化预处理
# 使用数据集映射函数将预处理函数应用于整个数据集
ccpm_train_tokenized = list(map(preproccess_function, ccpm_train_concat))
ccpm_valid_tokenized = list(map(preproccess_function, ccpm_valid_concat))

In [24]:
ccpm_train_tokenized

[{'input_ids': [101, 100, 1756, 100, 1989, 100, 100, 100, 100, 100, 100, 1916, 1802, 1863, 100, 1931, 100, 100, 1636, 1027, 1945, 1756, 100, 1742, 1875, 100, 100, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 100, 1756, 100, 1989, 100, 100, 100, 100, 100, 100, 1916, 1802, 1863, 100, 1931, 100, 100, 1636, 1027, 100, 1756, 100, 100, 1875, 100, 100, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 100, 1756, 100, 1989, 100, 100, 100, 100, 100, 100, 1916, 1802, 1863, 100, 1931, 100, 100, 1636, 1027, 1873, 1756, 100, 1742, 1875, 100, 100, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 100, 1756, 100, 1989, 100, 100, 100, 100, 100, 100, 1916, 1802, 1863, 100, 1931, 100, 100, 1636, 1027, 1945, 1756, 100, 100, 1875, 100, 100, 102], 'attention_m

In [31]:
ccpm_train_tokenized2 = []
for i in range(0, len(ccpm_train_concat)):
    temp_json = {"text": ccpm_train_concat[i]["text"],
                 "input_ids": ccpm_train_tokenized[i]["input_ids"],
                 "attention_mask":ccpm_train_tokenized[i]["attention_mask"],
                 "label": ccpm_train_concat[i]["label"]}
    ccpm_train_tokenized2.append(temp_json)

ccpm_valid_tokenized2 = []
for i in range(0, len(ccpm_valid_concat)):
    temp_json = {"text": ccpm_valid_concat[i]["text"],
                 "input_ids": ccpm_valid_tokenized[i]["input_ids"],
                 "attention_mask":ccpm_valid_tokenized[i]["attention_mask"],
                 "label": ccpm_valid_concat[i]["label"]}
    ccpm_valid_tokenized2.append(temp_json)

In [32]:
ccpm_train_tokenized2[0]

{'text': '诗人啊，你竟像在遥远的地方站立船头。=行人初上木兰舟',
 'input_ids': [101,
  100,
  1756,
  100,
  1989,
  100,
  100,
  100,
  100,
  100,
  100,
  1916,
  1802,
  1863,
  100,
  1931,
  100,
  100,
  1636,
  1027,
  1945,
  1756,
  100,
  1742,
  1875,
  100,
  100,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': 0}

In [33]:
# 4.文本动态填充到统一长度：最长元素的长度
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [34]:
# 训练
# 使用AutoModelForSequenceClassification加载模型以及语气标签的数量
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [49]:
%load_ext tensorboard
%tensorboard --logdir f"./results/runs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

In [38]:
from datasets import load_metric

metric = load_metric("accuracy")
metric.compute(predictions=[0,0,1,1], references=[0,1,1,1])

{'accuracy': 0.75}

In [44]:
# 在训练参数中定义训练超参数
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,

    evaluation_strategy="steps",
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="tensorboard"
)

PyTorch: setting up devices


In [45]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    results = metric.compute(predictions=predictions, references=labels)
    return results

In [46]:
# 将训练参数与模型、数据集、分词器和数据整理器一起传递给训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ccpm_train_tokenized2,
    eval_dataset=ccpm_valid_tokenized2,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [50]:
# 微调模型
trainer.train()

***** Running training *****
  Num examples = 87112
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5445
  Number of trainable parameters = 66955010
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 10880
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10880
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10880
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can sa

TrainOutput(global_step=5445, training_loss=0.5182401658855961, metrics={'train_runtime': 821.601, 'train_samples_per_second': 106.027, 'train_steps_per_second': 6.627, 'total_flos': 1149467223609504.0, 'train_loss': 0.5182401658855961, 'epoch': 1.0})

In [51]:
# 测试代码
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10880
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.5067731738090515,
 'eval_accuracy': 0.7667279411764706,
 'eval_runtime': 4.9951,
 'eval_samples_per_second': 2178.153,
 'eval_steps_per_second': 136.135,
 'epoch': 1.0}

In [21]:
# 模型测试
ccpm_test_tokenized = list(map(preproccess_function, ccpm_test_concat))

ccpm_test_tokenized2 = []
for i in range(0, len(ccpm_test_concat)):
    temp_json = {"text": ccpm_test_concat[i]["text"],
                 "input_ids": ccpm_test_tokenized[i]["input_ids"],
                 "attention_mask":ccpm_test_tokenized[i]["attention_mask"],
                 }
    ccpm_test_tokenized2.append(temp_json)

In [26]:
print(ccpm_test_tokenized2[0])

{'text': '侍奉夫主，不能尽自己的天年。=事主不尽年', 'input_ids': [101, 1765, 100, 1813, 1747, 1989, 1744, 100, 100, 100, 100, 1916, 1811, 1840, 1636, 1027, 1751, 1747, 1744, 100, 1840, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# for batch in ccpm_test_tokenized2:
#     print(batch)
#     print(trainer.predict(ccpm_test_tokenized2))
    # # ... (necessary processing)
    # logits = model(**batch, siz).logits
    # # ... (necessary post processing)
    # # ... (accumulate or compute the accuracy)

In [27]:
test_predictions = trainer.predict(ccpm_test_tokenized2)
test_predictions_argmax = np.argmax(test_predictions[0], axis=1)
test_references = np.array()
for i in range(0, len(ccpm_test_tokenized2)):
    test_references.append(ccpm_test_tokenized2[i]["label"])
metric.compute(predictions=test_predictions_argmax, references=test_references)

NameError: name 'trainer' is not defined